In [1]:
#Import packages
import os
import glob
import json
import pickle
import shutil
from pathlib import Path

#from tqdm import tqdm

import nilearn
import nibabel as nib
from nibabel import load
from nibabel.gifti import GiftiDataArray, GiftiImage

from nilearn import image as nimg
from nilearn import plotting as nplot
from nilearn.glm.first_level import FirstLevelModel, make_first_level_design_matrix, run_glm
from nilearn.glm.second_level import make_second_level_design_matrix, SecondLevelModel
from nilearn.glm import fdr_threshold,threshold_stats_img
from nilearn.glm.contrasts import compute_contrast, _compute_fixed_effects_params
from bids.layout import BIDSLayout, parse_file_entities

# import cortex
# from cortex import fmriprep

from nipype.interfaces.workbench.base import WBCommand
from nipype.algorithms import modelgen
from nipype.interfaces.base import Bunch

import scipy.stats as stats

import hcp_utils as hcp

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image

#my utilities package
import utils

#%matplotlib inline
#! module load openmind/hcp-workbench/1.2.3

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [2]:
#take all the first level files and parse out subject, session, and task name, remove repeats 

#first_level_root = f'../../derivatives/first_level_110123'

#folder = 'first_level_percondition'
#folder = 'first_level_emosent_ftest_112523'
#folder = 'first_level_emosent_emospeech_112523'
# folder = 'first_level_emosent_sadneutral_112713'
# first_level_root = f'../../derivatives/{folder}/'
folder = 'test'
first_level_root = 'test/'
#first_level_root = '../../derivatives/first_level_main_112223/'
first_level_effects = glob.glob(f'{first_level_root}/*effect_size*.dscalar.nii')
effect_variances = glob.glob(f'{first_level_root}/*effect_variance*.dscalar.nii')

#load list of runs to exclude and format as a list of bids-compliant names
# exclude_df = pd.read_table('../../derivatives/first_level_110123/no_wall_of_speech_exclude_110123 - Sheet1.tsv')

# exclude = []
# for i, r in exclude_df.iterrows():
#     sub = r['sub']
#     ses = r['ses']
#     task = r.iloc[5]
#     run = r['run']
#     exclude.append(f'sub-voice{sub}_ses-{ses}_task-{task}_rec-unco_run-{run}')

    
    
#remove runs from exclusion list
effects_qc = [beta for beta in first_level_effects] 
#                       if beta.split(f'{folder}/')[1].split('_space')[0] not in exclude]
variance_qc = [beta for beta in effect_variances]
#                       if beta.split(f'{folder}/')[1].split('_space')[0] not in exclude]



#get final list of sub/ses/tasks we want after fixed effects pooling
#sub_ses_task_wdup = [sn.split(f'{folder}/')[1].split('_rec-unco')[0] for sn in effects_qc]
sub_ses_task_wdup = [(sn.split(f'{folder}/')[1].split('_rec-unco')[0], sn.split('_den-91k_')[1].split('_effect_')[0]) 
                     for sn in effects_qc]
sub_ses_task = []
[sub_ses_task.append(x) for x in sub_ses_task_wdup if x not in sub_ses_task]

#keep only the tasks we want
#task_list = ['nwr','emosent']
task_list = ['emosent']
con = 'speechvsil'
#con = 'happyvneutral'
#task_list = ['pataka', 'emosent', 'vowel', 'nwr']
sub_ses_task = [sst for task in task_list for sst in sub_ses_task  if task in sst[0]] 
sub_ses_task = [sst for sst in sub_ses_task  if con in sst[1]] 

len(sub_ses_task), len(effects_qc), len(variance_qc), sub_ses_task[0:5]

(5,
 10,
 10,
 [('sub-voice856_ses-1_task-emosent', 'contrast-speechvsil'),
  ('sub-voice859_ses-1_task-emosent', 'contrast-speechvsil'),
  ('sub-voice860_ses-1_task-emosent', 'contrast-speechvsil'),
  ('sub-voice875_ses-1_task-emosent', 'contrast-speechvsil'),
  ('sub-voice893_ses-1_task-emosent', 'contrast-speechvsil')])

In [7]:
#!rm test/second_level/*
!ls test/second_level/

sub-voice856_ses-1_task-emosent_rec-unco_space-fsLR_den-91k_contrast-speechvsil_effect_size_fx.dscalar.nii
sub-voice856_ses-1_task-emosent_rec-unco_space-fsLR_den-91k_contrast-speechvsil_effect_variance_fx.dscalar.nii
sub-voice856_ses-1_task-emosent_rec-unco_space-fsLR_den-91k_contrast-speechvsil_stat_fx.dscalar.nii
sub-voice859_ses-1_task-emosent_rec-unco_space-fsLR_den-91k_contrast-speechvsil_effect_size_fx.dscalar.nii
sub-voice859_ses-1_task-emosent_rec-unco_space-fsLR_den-91k_contrast-speechvsil_effect_variance_fx.dscalar.nii
sub-voice859_ses-1_task-emosent_rec-unco_space-fsLR_den-91k_contrast-speechvsil_stat_fx.dscalar.nii
sub-voice860_ses-1_task-emosent_rec-unco_space-fsLR_den-91k_contrast-speechvsil_effect_size_fx.dscalar.nii
sub-voice860_ses-1_task-emosent_rec-unco_space-fsLR_den-91k_contrast-speechvsil_effect_variance_fx.dscalar.nii
sub-voice860_ses-1_task-emosent_rec-unco_space-fsLR_den-91k_contrast-speechvsil_stat_fx.dscalar.nii
sub-voice875_ses-1_task-emosent_rec-unco_space

In [6]:
#out_dir = '../../derivatives/second_level_main_112223'
#out_dir = '../../derivatives/second_level_percondition_112123'
#out_dir = '../../derivatives/second_level_emosent_ftest_112523'
#out_dir = '../../derivatives/second_level_emosent_emospeech_112523'
#out_dir = '../../derivatives/second_level_emosent_sadneutral_112713'
out_dir = 'test/second_level'


for info in sub_ses_task:
    
    #filter out sub, ses, task, and contrast we want
    effect_size_list = [file for file in effects_qc if info[0] in file]
    effect_size_list = [file for file in effect_size_list if info[1] in file]
    
    effect_var_list = [file for file in variance_qc if info[0] in file]
    effect_var_list = [file for file in effect_var_list if info[1] in file]

    if effect_size_list:

        # no fixed effects if only 1 run for a subject, copy first level betas to second level folder
        if len(effect_size_list) == 1:
            shutil.copy(effect_size_list[0], os.path.join(out_dir, str(Path(effect_size_list[0]).relative_to(first_level_root))))
            shutil.copy(effect_var_list[0], os.path.join(out_dir, str(Path(effect_var_list[0]).relative_to(first_level_root))))
        
        # pool multiple runs from one subject with fixed effects and save outputs
        elif len(effect_size_list) > 1:                
            fx_results = _compute_fixed_effects_params(
                np.squeeze(
                     [nib.load(fname).get_fdata(dtype='f4') for fname in effect_size_list]
                 ),
                 np.squeeze(
                     [nib.load(fname).get_fdata(dtype='f4') for fname in effect_var_list]
                 ),
                 precision_weighted=False)
            
            #save the outputs
            #note the order of fixed effects outputs: fx_results = [fixed_fx_contrast, fixed_fx_variance, fixed_fx_tstat]            
            utils.save_cifti(fx_results[0], 'effect_size_fx', out_dir, info[0] + '_rec-unco', contrast=info[1].split('contrast-')[1])
            utils.save_cifti(fx_results[1], 'effect_variance_fx', out_dir, info[0] + '_rec-unco',contrast=info[1].split('contrast-')[1])
            utils.save_cifti(fx_results[2], 'stat_fx', out_dir, info[0] + '_rec-unco',contrast=info[1].split('contrast-')[1])

In [8]:
[nib.load(fname).get_fdata(dtype='f4') for fname in effect_size_list]

[memmap([[-12.714197, -25.39303 , 254.86409 , ...,  20.513195,  38.321392,
           47.17918 ]], dtype=float32),
 memmap([[-35.603855 ,  19.969484 , 275.52957  , ...,  37.12193  ,
           -2.1463413,   7.1514325]], dtype=float32)]

In [7]:
np.squeeze(
                     [nib.load(fname).get_fdata(dtype='f4') for fname in effect_size_list]
                 ).shape

(2, 91282)